In [22]:
#definition of the yaw cases

yaw_cases = {
    "0deg_baseline_csv": 0.0,
    "15deg_baseline_csv": 15.0,
    "30deg_baseline_csv": 30.0,
    "45deg_baseline_csv": 45.0,
}
import os 
import numpy as np 
import pandas as pd


In [23]:
def read_field_csv(path, expected_components, field_name):
    """
    Read a CSV file for a scalar or vector field and return a numeric numpy array.

    expected_components = 1 for scalar (p, k, nut, omega)
                        = 3 for vector (U)
    """
    # 1) Read with header row
    df = pd.read_csv(path, header=0)   # first line is header: 'cellId,Ux,Uy,Uz' etc.

    # 2) Convert all columns to numeric, coercing bad values to NaN
    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # 3) Drop columns that are completely NaN (purely non-numeric or empty)
    df = df.dropna(axis=1, how='all')

    # 4) Drop rows that are completely NaN (e.g., blank lines at end)
    df = df.dropna(axis=0, how='all')

    # 5) Now we should only have numeric columns.
    #    If there are extra columns (e.g., 'cellId'), keep only the last expected_components.
    if df.shape[1] > expected_components:
        df = df.iloc[:, -expected_components:]

    # 6) If there are fewer columns, that’s a real problem.
    if df.shape[1] != expected_components:
        raise ValueError(
            f"{field_name}: expected {expected_components} numeric columns, "
            f"but got {df.shape[1]} in file {path}. "
            f"Columns seen: {list(df.columns)}"
        )

    return df.values

In [24]:

def load_snapshot(folder_path, n_cells_ref=None):
    """Load one timestep folder: U, p, k, nut, omega; flatten into 1D vector."""

    U = read_field_csv(
        os.path.join(folder_path, "U.csv"),
        expected_components=3,
        field_name="U"
    )  # shape: (N, 3)

    p = read_field_csv(
        os.path.join(folder_path, "p.csv"),
        expected_components=1,
        field_name="p"
    )  # shape: (N, 1)

    k = read_field_csv(
        os.path.join(folder_path, "k.csv"),
        expected_components=1,
        field_name="k"
    )

    nut = read_field_csv(
        os.path.join(folder_path, "nut.csv"),
        expected_components=1,
        field_name="nut"
    )

    omega = read_field_csv(
        os.path.join(folder_path, "omega.csv"),
        expected_components=1,
        field_name="omega"
    )

    # Consistency check: same number of cells in all fields
    N = U.shape[0]
    for arr, name in [(p, "p"), (k, "k"), (nut, "nut"), (omega, "omega")]:
        if arr.shape[0] != N:
            raise ValueError(
                f"Cell count mismatch in {folder_path}: "
                f"U has {N} rows but {name} has {arr.shape[0]} rows"
            )

    # Optional: check against reference cell count from the first snapshot
    if n_cells_ref is not None and N != n_cells_ref:
        raise ValueError(
            f"Inconsistent cell count in {folder_path}: "
            f"expected {n_cells_ref}, but got {N}"
        )

    # Concatenate along feature dimension and flatten
    fields = np.hstack([U, p, k, nut, omega])  # shape: (N, 7)
    flat = fields.flatten()                    # shape: (N*7,)

    return flat, N


In [25]:

def load_yaw_case(base_dir, yaw_angle):
    """Load all snapshots for one yaw case, enforcing consistent shape."""
    X_list = []
    yaw_list = []

    time_dirs = sorted([d for d in os.listdir(base_dir) if d.isdigit()])

    n_cells_ref = None

    for t in time_dirs:
        snapshot_dir = os.path.join(base_dir, t)
        flat_vec, N = load_snapshot(snapshot_dir, n_cells_ref)

        if n_cells_ref is None:
            n_cells_ref = N  # first snapshot defines reference

        X_list.append(flat_vec)
        yaw_list.append(yaw_angle)

    # Now everything should have identical length → safe to stack
    X_arr = np.stack(X_list, axis=0)                 # shape: (num_times, D)
    yaw_arr = np.array(yaw_list, dtype=float).reshape(-1, 1)  # shape: (num_times, 1)

    return X_arr, yaw_arr, n_cells_ref


In [26]:
yaw_folders = {
    "0deg_baseline_csv": 0.0,
    "15deg_baseline_csv": 15.0,
    "30deg_baseline_csv": 30.0,
    "45deg_baseline_csv": 45.0,
}

X_all = []
yaw_all = []
n_cells_each_case = {}

base_path = "../"  # adjust as needed

for folder, yaw_angle in yaw_folders.items():
    full_path = os.path.join(base_path, folder)
    Xi, yi, N = load_yaw_case(full_path, yaw_angle)

    print(f"{folder}: {Xi.shape[0]} snapshots, {N} cells per snapshot")

    n_cells_each_case[yaw_angle] = N
    X_all.append(Xi)
    yaw_all.append(yi)

X_all = np.vstack(X_all)
yaw_all = np.vstack(yaw_all)

print("Dataset shape:", X_all.shape)
print("Yaw labels shape:", yaw_all.shape)
print("Cells per case:", n_cells_each_case)


ValueError: Inconsistent cell count in ../0deg_baseline_csv/100: expected 1, but got 162506

In [ ]:
from sklearn.decomposition import PCA

latent_dim = 50  # try 20–100

pca = PCA(n_components=latent_dim)
Z = pca.fit_transform(X_all)

print("Latent Z shape:", Z.shape)


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

class YawToLatent(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(1, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, latent_dim)
        )

    def forward(self, yaw):
        return self.net(yaw)


model = YawToLatent(latent_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

dataset = TensorDataset(torch.tensor(yaw_all, dtype=torch.float32),
                        torch.tensor(Z, dtype=torch.float32))

loader = DataLoader(dataset, batch_size=8, shuffle=True)

for epoch in range(500):
    for yaw_batch, z_batch in loader:
        optimizer.zero_grad()
        z_pred = model(yaw_batch)
        loss = loss_fn(z_pred, z_batch)
        loss.backward()
        optimizer.step()

    if epoch % 50 == 0:
        print("Epoch", epoch, "loss", loss.item())


In [ ]:
yaw_query = torch.tensor([[22.5]], dtype=torch.float32)

Z_pred = model(yaw_query).detach().numpy()
X_pred = pca.inverse_transform(Z_pred)[0]


In [ ]:
num_features = 3 + 1 + 1 + 1 + 1  # U + p + k + nut + omega
N_cells = X_all.shape[1] // num_features

pred_fields = X_pred.reshape(N_cells, num_features)

Ux = pred_fields[:, 0]
Uy = pred_fields[:, 1]
Uz = pred_fields[:, 2]
p  = pred_fields[:, 3]
k  = pred_fields[:, 4]
nut= pred_fields[:, 5]
omega=pred_fields[:, 6]
